In [1]:
# https://github.com/natasha/navec

from navec import Navec
from razdel import tokenize, sentenize
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from funcs import *

navec = Navec.load('navec_hudlit_v1_12B_500K_300d_100q.tar')

In [2]:
df = pd.read_csv('test_corpus1.csv')
df = df[df['text'].notnull()]
sentiment_mapping = {'bad': 0, 'neutral': 0, 'good': 1}
df['target'] = df['coloring'].map(sentiment_mapping)
df['target'].value_counts()

1    2120
0    1996
Name: target, dtype: int64

## Слова в данной библиотеке кодируются следующим образом.

## Убедимся, что король - мужчина + женщина = королева

In [3]:
def print_nearest(vector, matrix, n=10):
    nearest_i = np.argsort(np.sum((matrix - vector)**2, axis=1))[:n]
    nearest = [navec.vocab.words[i] for i in nearest_i]
    print(nearest)

# navec.pq[navec.vocab.words.index(word)] == navec[word]
word = 'стол'
word_index = navec.vocab.words.index(word)
word_vec = navec[word]
print(f'Веторное представление слова "стол": {word_vec[:10]}')
navec_np = np.array([list(navec[word]) for word in navec.vocab.words])

Веторное представление слова "стол": [-0.2938254  -0.2968786  -0.02534253 -0.19875844  0.2802261  -0.14141266
  0.05130551 -0.18968578  0.728676   -0.6062129 ]


## Данная функция находит ближайшие слова к заданному вектору

In [4]:
print_nearest(navec['стол'], navec_np)
print_nearest(navec['король'], navec_np)
print_nearest(navec['король'] - navec['мужчина'] + navec['женщина'], navec_np)


['стол', 'стола', 'столу', 'столик', 'стул', 'столе', 'диван', 'поднос', 'столом', 'кухонный']
['король', 'принц', 'герцог', 'короля', 'генрих', 'людовик', 'монарх', 'величество', 'королю', 'император']
['король', 'королева', 'принцесса', 'короля', 'изабелла', 'королю', 'герцогиня', 'величество', 'людовик', 'королева-мать']


## Посмотрим, как работает токенизатор:

In [5]:
text = df.iloc[10, 1]
print(text)
list(tokenize(text))[:20]

Добрый день, уважаемые пользователи банковских услуг. Хочу предупредить что в данном банке не существуют единых (стандартов/гостов), сотрудники не квалифицированные на горячей линии, получить интересующую информацию возможно только в отделении банка где открыт счёт компании. Регалии современного века ввиде электронного оборота, банк не поддерживает.  Не компетентность банковских сотрудников приносит существенный урон бизнесу. Всю информацию запрашиваемую у банка, лучше получать в чате, что бы было на руках подтверждение всего деалога.


[Substring(0, 6, 'Добрый'),
 Substring(7, 11, 'день'),
 Substring(11, 12, ','),
 Substring(13, 22, 'уважаемые'),
 Substring(23, 35, 'пользователи'),
 Substring(36, 46, 'банковских'),
 Substring(47, 52, 'услуг'),
 Substring(52, 53, '.'),
 Substring(54, 58, 'Хочу'),
 Substring(59, 71, 'предупредить'),
 Substring(72, 75, 'что'),
 Substring(76, 77, 'в'),
 Substring(78, 84, 'данном'),
 Substring(85, 90, 'банке'),
 Substring(91, 93, 'не'),
 Substring(94, 104, 'существуют'),
 Substring(105, 111, 'единых'),
 Substring(112, 113, '('),
 Substring(113, 123, 'стандартов'),
 Substring(123, 124, '/')]

## Векторизивывать текст будем просто за счёт суммирования векторов всех слов, которые в него входят. Для классификации будем использвать линейную регрессию.

In [6]:
df['vector'] = df['text'].map(vectorize_text)
df_train, df_test = train_test_split(df[['text', 'target', 'coloring', 'vector']].copy(), test_size=0.3)
x_train, x_test = np.array(df_train['vector'].to_list()), np.array(df_test['vector'].to_list())
y_train, y_test = df_train['target'], df_test['target']
clf = LogisticRegression(multi_class='ovr', class_weight=(1/df_train['target'].value_counts()).to_dict(), max_iter=10000)
clf.fit(x_train, y_train)

LogisticRegression(class_weight={0: 0.0007309941520467836,
                                 1: 0.0006609385327164573},
                   max_iter=10000, multi_class='ovr')

## В качестве метрики качества будем использовать ROC AUC

In [7]:
y_pred_train, y_pred_test = clf.predict_proba(x_train), clf.predict_proba(x_test)
df_train.insert(0, 'predictions', [x.round(3).tolist() for x in y_pred_train])
df_test.insert(0, 'predictions', [x.round(3).tolist() for x in y_pred_test])
df_test['predictions'] = [x.round(3).tolist() for x in y_pred_test]
y_pred_train_label, y_pred_test_label = y_pred_train.argmax(axis=1), y_pred_test.argmax(axis=1)
print(multiclass_roc_auc(y_train, y_pred_train))
print(multiclass_roc_auc(y_test, y_pred_test))

0.9696997367841281
0.9503903503709378


## Confusion матрица

In [8]:
confusion = pd.DataFrame({'true': y_test, 'pred': y_pred_test_label}).groupby(['true', 'pred']).size() \
    .reset_index(name='fraction').pivot(index='true', columns='pred', values='fraction') \
    .reindex(index=set(sentiment_mapping.values()), columns=set(sentiment_mapping.values())).fillna(0)
confusion /= confusion.sum().sum()
print(confusion)

pred         0         1
true                    
0     0.439676  0.068826
1     0.041296  0.450202


In [9]:
print(y_pred_train)
print(y_pred_test)

[[0.19668237 0.80331763]
 [0.3945243  0.6054757 ]
 [0.05193555 0.94806445]
 ...
 [0.19154803 0.80845197]
 [0.88571342 0.11428658]
 [0.66092604 0.33907396]]
[[1.14607568e-01 8.85392432e-01]
 [1.90407532e-01 8.09592468e-01]
 [9.85784216e-01 1.42157835e-02]
 ...
 [5.03024567e-02 9.49697543e-01]
 [9.99992476e-01 7.52364105e-06]
 [8.66838263e-01 1.33161737e-01]]


##  Посмотрим не те тексты, которые были наиболее неправильно классифицированны

In [10]:
print_most_incorrect(df_train)
print_most_incorrect(df_test)

('Добрый день! Решился на написание отзыва после двухнедельного отфутболивания '
 'сотрудниками банка. Некомпетентность сотрудников call-центра и офисов банка '
 'просто не знает границ.  Начну история сначала: Брали ипотечный кредит в ЦИК '
 '"Смоленский" офис ВТБ в Москве. Сразу поясню: Квартира находится в Москве, а '
 'сами мы проживаем в Белгороде. Квартира еще строится. Брали ее по ДДУ.</li> '
 'Досрочно осуществили полное погашение ипотечного кредита через год и тут все '
 'началось...</li> После погашения ипотечного кредита решили поднять вопрос о '
 'снятии обременения с квартиры. Ехать в Мск для снятия обременения не '
 'хотелось, так как в других банках все снимают автоматически и удаленно (без '
 'участия клиента), например, в Сбербанке, то была мысль, что и тут делается '
 'все также.</li> Позвонили в call-центр банка и пообщались с двумя '
 'операторами, которые изначально переадресовывали вопрос друг на друга, а '
 'потом сказали, что оставят заявку в спец. отделе по ипо